In [1]:
import pandas as pd

train_df = pd.read_csv('train.csv').drop(columns=['id'])
train_df = train_df.dropna(subset=['Number_of_Ads'])
print(train_df.shape)
print(train_df.columns)
train_df.head()

(749999, 11)
Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes'],
      dtype='object')


,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [2]:
# Split the data into train and test sets randomly and stratified by the target variable
from sklearn.model_selection import train_test_split

sub_train, sub_test = train_test_split(train_df, test_size=0.2, random_state=42)

In [3]:
# get the target variable and features
X_sub_train = sub_train.drop(columns=['Listening_Time_minutes'])
y_sub_train = sub_train['Listening_Time_minutes']
X_sub_test = sub_test.drop(columns=['Listening_Time_minutes'])
y_sub_test = sub_test['Listening_Time_minutes']
print(X_sub_train.shape, y_sub_train.shape)
print(X_sub_test.shape, y_sub_test.shape)

(599999, 10) (599999,)
(150000, 10) (150000,)


In [4]:
num_attributes = X_sub_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_attributes = X_sub_train.select_dtypes(include=['object']).columns.tolist()

num_attributes

['Episode_Length_minutes',
 'Host_Popularity_percentage',
 'Guest_Popularity_percentage',
 'Number_of_Ads']

In [5]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class GroupMedianImputer(BaseEstimator, TransformerMixin):
    """
    Imputes missing values in col_to_impute by taking
    the median of that column within groups defined by group_cols.
    """
    def __init__(self, group_cols, col_to_impute):
        self.group_cols = group_cols
        self.col_to_impute = col_to_impute
        self.group_medians_ = None
        self.global_median_ = None

    def fit(self, X, y=None):
        # Convert to DataFrame if needed
        df = pd.DataFrame(X.copy())
        
        # Compute groupwise median
        self.group_medians_ = (
            df.groupby(self.group_cols)[self.col_to_impute]
              .median()
              .to_dict()
        )
        # Compute overall median as fallback
        self.global_median_ = df[self.col_to_impute].median()
        return self

    def transform(self, X, y=None):
        df = pd.DataFrame(X.copy())
        
        # We'll apply groupwise median for each row
        def fill_with_group_median(row):
            group_key = tuple(row[col] for col in self.group_cols)
            if pd.isna(row[self.col_to_impute]):
                # if group exists in dictionary, use that median
                # otherwise, use global median
                return self.group_medians_.get(group_key, self.global_median_)
            else:
                return row[self.col_to_impute]
        
        df[self.col_to_impute] = df.apply(fill_with_group_median, axis=1)
        
        # Fill any remaining missing with global median
        df[self.col_to_impute].fillna(self.global_median_, inplace=True)
        
        return df



In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


col_names = num_attributes

#find the column number of the column 'Episode_Length_minutes', 'Guest_Popularity_percentage', 'Host_Popularity_percentage'
episode_length_index = col_names.index('Episode_Length_minutes')
guest_popularity_index = col_names.index('Guest_Popularity_percentage')
host_popularity_index = col_names.index('Host_Popularity_percentage')

class NewAttributesAdder(BaseEstimator, TransformerMixin):
    """
    Adds new attributes to the DataFrame.
    """
    def __init__(self, add_new_attributes=1):
        self.add_new_attributes = add_new_attributes
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
       
        ideal_listening_time_guest1 = X[:, episode_length_index] * X[:, host_popularity_index] * 0.01

        ideal_listening_time_guest2 = X[:, episode_length_index] * X[:, guest_popularity_index] * 0.01

        # Add the new attributes to the DataFrame
        ideal_listening_time_guest3 = X[:, episode_length_index] * (X[:, host_popularity_index] * X[:, guest_popularity_index] * 0.0001)

        return np.c_[X, ideal_listening_time_guest1, ideal_listening_time_guest2]
    





# Model 5 Use median of global data set to impute.

Also add new attribute that popularity 

In [7]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
# Create a pipeline with the custom and simple imputers

from sklearn.impute import KNNImputer

# Assume X is your feature matrix
knn_imputer = KNNImputer(n_neighbors=5)



group_imputer = GroupMedianImputer(
    group_cols=['Podcast_Name', 'Episode_Title'], 
    col_to_impute='Episode_Length_minutes'
)

group_imputer2 = GroupMedianImputer(
    group_cols=['Podcast_Name', 'Episode_Title'], 
    col_to_impute='Guest_Popularity_percentage'
)

simple_imputer = SimpleImputer(strategy='median')


imputer_pipeline = Pipeline(
    steps=[
        ('group_imputer', group_imputer),
        ('group_imputer2', group_imputer2)
    ]
)


col_names = X_sub_train.columns.tolist()


attr_adder = NewAttributesAdder(add_new_attributes=1)


num_attributes = X_sub_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_attributes = X_sub_train.select_dtypes(include=['object']).columns.tolist()

print(num_attributes)

print(cat_attributes)

num_pipeline = Pipeline([
    ('minmax', MinMaxScaler()),
    ('imputer', simple_imputer),
    ('attr_adder', attr_adder)
])

num_pipeline2 = Pipeline([
    ('minmax', RobustScaler()),
    ('imputer', simple_imputer),
    ('attr_adder', attr_adder)
])


cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder())
])


combine_pipeline = ColumnTransformer([
    ('scaler', num_pipeline, num_attributes),
    ('cat', cat_pipeline, cat_attributes)
])

combine_pipeline2 = ColumnTransformer([
    ('scaler', num_pipeline2, num_attributes),
    ('cat', cat_pipeline, cat_attributes)
])

# Fit the imputer pipeline on the training data
full_pipeline = Pipeline(steps=[
    ('imputer', imputer_pipeline),
    ('combine', combine_pipeline)
])

['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads']
['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']


In [8]:
train_df_grouped_imputed = combine_pipeline2.fit_transform(X_sub_train)

In [9]:
print(train_df_grouped_imputed[0])

  (0, 0)	0.07788642991936867
  (0, 1)	0.24707055597107963
  (0, 2)	0.6570718575274385
  (0, 3)	-0.5
  (0, 4)	0.00019243443542780948
  (0, 5)	0.0005117698118330023
  (0, 42)	1.0
  (0, 149)	1.0
  (0, 163)	1.0
  (0, 165)	1.0
  (0, 174)	1.0
  (0, 175)	1.0


In [10]:
def display_scores(scores):
    scores = np.sqrt(-scores)  # Convert to RMSE
    #print the variable name
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [11]:
# Cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

import numpy as np


lin_reg = LinearRegression()

# Evaluate the models using cross-validation

lin_scores = cross_val_score(lin_reg, train_df_grouped_imputed, 
                             y_sub_train, scoring='neg_mean_squared_error', cv=5)

    
display_scores(lin_scores)

Scores: [13.43301682 13.33173052 13.4043473  13.30091122 13.33735673]
Mean: 13.361472516266968
Standard deviation: 0.04917451003026039


In [12]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1, random_state=42)

lasso_scores = cross_val_score(lasso_reg, train_df_grouped_imputed, 
                               y_sub_train, scoring='neg_mean_squared_error', cv=5)

display_scores(lasso_scores)

Scores: [13.46065267 13.36973227 13.4378435  13.33513602 13.37757936]
Mean: 13.396188762864629
Standard deviation: 0.04618312783445473


In [13]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.1, random_state=42)
ridge_scores = cross_val_score(ridge_reg, train_df_grouped_imputed, 
                                 y_sub_train, scoring='neg_mean_squared_error', cv=5)
display_scores(ridge_scores)




Scores: [13.43322703 13.33259974 13.40440834 13.30128437 13.33733981]
Mean: 13.361771859824206
Standard deviation: 0.04905177921692511


In [14]:
# # Add grid search for hyperparameter tuning
# from sklearn.model_selection import GridSearchCV

# param_grid = [
#     {'alpha': [0.1, 1, 10]}
# ]
# ridge_reg = Ridge(random_state=42)
# grid_search = GridSearchCV(ridge_reg, param_grid, cv=5,
#                              scoring='neg_mean_squared_error')
# grid_search.fit(train_df_grouped_imputed, y_sub_train)

# print("Best parameters:", grid_search.best_params_)

from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=10, random_state=42)
ridge_scores = cross_val_score(ridge_reg, train_df_grouped_imputed, 
                                 y_sub_train, scoring='neg_mean_squared_error', cv=5)
display_scores(ridge_scores)



Scores: [13.43846967 13.3382359  13.41050901 13.30642665 13.34237906]
Mean: 13.367204057662061
Standard deviation: 0.04920047768678487


In [15]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_scores = cross_val_score(xgb_reg, train_df_grouped_imputed, 
                               y_sub_train, scoring='neg_mean_squared_error', cv=3)
display_scores(xgb_scores)

Scores: [13.10816152 13.12456785 13.06170055]
Mean: 13.098143309939745
Standard deviation: 0.026625148016640385


In [21]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=7, random_state=42)
xgb_scores = cross_val_score(xgb_reg, train_df_grouped_imputed, 
                               y_sub_train, scoring='neg_mean_squared_error', cv=3)
display_scores(xgb_scores)

Scores: [13.02028735 13.0430344  12.97324611]
Mean: 13.012189285551178
Standard deviation: 0.029060684204455452


In [23]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(n_estimators=500, learning_rate=0.08, max_depth=11, random_state=42)
xgb_scores = cross_val_score(xgb_reg, train_df_grouped_imputed, 
                               y_sub_train, scoring='neg_mean_squared_error', cv=3)
display_scores(xgb_scores)

Scores: [12.94723877 12.95584414 12.88265409]
Mean: 12.928579000272224
Standard deviation: 0.03266329340100232


In [25]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(n_estimators=500, learning_rate=0.04, max_depth=15, random_state=42)
xgb_scores = cross_val_score(xgb_reg, train_df_grouped_imputed, 
                               y_sub_train, scoring='neg_mean_squared_error', cv=3)
display_scores(xgb_scores)

Scores: [12.8985731  12.90896444 12.83945082]
Mean: 12.882329454228673
Standard deviation: 0.030615113578508605


In [26]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [500],
    'max_depth': [15, 19],
    'learning_rate': [0.01, 0.04]
}

grid_search = GridSearchCV(XGBRegressor(random_state=42), 
                           param_grid, 
                           cv=3,
                           scoring='neg_mean_squared_error')  # or another metric relevant to your problem
grid_search.fit(train_df_grouped_imputed, y_sub_train)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


Best parameters: {'learning_rate': 0.04, 'max_depth': 15, 'n_estimators': 500}
Best score: -165.955349452467


In [16]:
from sklearn.ensemble import StackingRegressor


estimators = [
    ('LR', LinearRegression()),
    ('xgb', XGBRegressor(random_state=42, n_estimators=100)),
]

# Replace LogisticRegression with a regressor, e.g., LinearRegression
stacking_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
scores = cross_val_score(stacking_model, train_df_grouped_imputed,
                                 y_sub_train, scoring='neg_mean_squared_error', cv=3)

display_scores(scores)

Scores: [13.07659115 13.09490806 13.02825443]
Mean: 13.066584547161204
Standard deviation: 0.02811613663105719


In [17]:
from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor


estimators = [
    ('LR', LinearRegression()),
    ('xgb', XGBRegressor(random_state=42, n_estimators=100)),
    ('DecisionTree', DecisionTreeRegressor(min_samples_split=1000, random_state=42)),
]

# Replace LogisticRegression with a regressor, e.g., LinearRegression
stacking_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
scores = cross_val_score(stacking_model, train_df_grouped_imputed,
                                 y_sub_train, scoring='neg_mean_squared_error', cv=3)

display_scores(scores)

Scores: [13.06126285 13.08255711 13.01853024]
Mean: 13.054116729334504
Standard deviation: 0.026622800446422352


In [18]:
from sklearn.ensemble import StackingRegressor


estimators = [
    ('LR', LinearRegression()),
    ('xgb', XGBRegressor(random_state=42, n_estimators=100)),
    ('ridge', Ridge(alpha=0.1, random_state=42)),
    ('lasso', Lasso(alpha=0.1, random_state=42)),
]

# Replace LogisticRegression with a regressor, e.g., LinearRegression
stacking_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
scores = cross_val_score(stacking_model, train_df_grouped_imputed,
                                 y_sub_train, scoring='neg_mean_squared_error', cv=3)

display_scores(scores)


Scores: [13.07673201 13.09486604 13.02856462]
Mean: 13.066720890268227
Standard deviation: 0.02797780947269261
